<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<br><br><br>
<h2><font color="#00586D" size=5>Capstone II: Recolección, preparación y almacenamiento de datos</font></h2>



<h1><font color="#00586D" size=6>Elaboración de una base de datos sobre películas.</font></h1>

<br><br>
<div style="text-align: right">
<font color="#00586D" size=3>Jacinto Arias, Javier Cózar, Juan Ignacio Alonso</font><br>
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube I </font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>

---

<br>

Este proyecto servirá para poner en práctica las herramientas y métodos que han sido tratados durante el segundo módulo del curso. 

Trabajaremos con diversas fuentes de datos de películas y extraeremos la información usando distintas técnicas:

- IMDB: La famosa página de bases de datos de películas, extraeremos datos de un clon usando web scraping
- IMDB: Trabajaremos con sus volcados de datos para hacer un proceso de fusión de la información
- TMDB: Utilizaremos esta popular API para extraer información que complemente a la anterior.

Por último integraremos parte de la información en una base de datos de MongoDB con la que podremos implementar un buscador de películas

<a id="section1"></a>
## <font color="#00586D"> 1. Lista de películas de IMDb  </font>
<br>

_Internet Movie Database ( [IMDb](http://www.imdb.com) )_ es uno de los sitios de referencia sobre películas y series en internet. Entre otras cosas, almacena información sobre repartos, tramas, presupuestos, etc. Además, contiene numerosas valoraciones y revisiones proporcionadas por los usuarios. Toda esta información es potencialmente accesible y recuperable de manera automática mediante __scraping__, aunque  __IMDb prohibe expresamente este tipo de acceso__, a menos que se cuente con una autorización. 

### <font color="#00586D"> 1.1 Web scraping del dump de IMDB. </font>
<br>

Debido a la imposibilidad de obtener la autorización para este tipo de proyectos, vamos a trabajar sobre un pequeño *dump* de IMDB, disponible en la dirección http://c3-capstone2.jialonso.com. Este *dump* contiene las 10 películas mejor valoradas en IMDB y es una versión reducida del listado de 250 mejores películas disponible en https://www.imdb.com/chart/top.

En primer lugar, importamos las librerías que utilizaremos para hacer web scraping.

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

La página inicial contiene el listado de películas y el enlace a la página de cada una de ellas. Lo primero que vamos a hacer es obtener una lista con todos los enlaces.

In [ ]:
def get_links(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    
    match = re.search('(https?://.+?)/', url)
    if match:
        base_url = match.group(1)
    else:
        base_url = url
        
    link_tags = soup.select('td.titleColumn a')
    return [f'{base_url}{a_tag.get("href")}' for a_tag in link_tags]
    
links = get_links('http://c3-capstone2.jialonso.com')
print(links)

El siguiente paso es obtener la información básica de cada una de las películas. Para cada una de las películas queremos obtener un diccionario con el siguiente formato:

    {
        'id': 'tt0068646',
        'title': {
            'primary': 'El padrino',
            'original': 'The Godfather'
        },
        'year': 1972,
        'genres': ['Crime', 'Drama'],
        'runtimeMinutes': 175
    }
    
Esta información se puede obtener de la cabecera de la película en la web, tal y como se muestra en la siguiente imagen:

<a id="inicio"></a>
<img src="img/el_padrino.png" alt="El padrino webscraping">

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 1</b></font>

Implementar la función `duration_string_to_minutes` que transforme strings del tipo 'Xh YYmin' (por ejemplo 2h 24min) en la duración en minutos (por ejemplo, 144). Para ello, **utilizar una expresión regular** que detecte el número de minutos y horas del string. Por simplicidad, puede suponerse que la duración mínima de las películas es de 1 hora.

In [ ]:
def duration_string_to_minutes(duration):
    """
    Convierte el string duration en el entero minutos. 
    Por ejemplo: duration_string_to_minutes("1h 37m") = 97
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
assert duration_string_to_minutes('2h 30min') == 150
assert duration_string_to_minutes('2h') == 120

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 2</b></font>

Implementar la función `get_movie_info` que dada la url de una película en el *dump* de IMDB obtenga el id, título primario y original, año, género(s) y duración. El resultado devuelto será un diccionario con el formato presentado anteriormente.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Algunas funciones de la librería BeautifulSoup admiten el uso de expresiones regulares. Por ejemplo `parent_tag.find_all('a', href=re.compile(r'.*foo.*'))` devolvería todos los tags `a` dentro del tag `parent_tag` que contienen el texto foo.
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: En ocasiones queremos obtener el texto incluído en un tag, pero no en sus descendientes, para ello podemos usar el método `.find(text=True)`. Por ejemplo:

    soup = BeautifulSoup('<div>foo<span>bar</span></div>')
    soup.get_text()  # foobar
    soup.find(text=True)  # foo

In [ ]:
def get_movie_info(url):
    """
    Devuelve los datos de la película de la URL dada como parámetro:
    
    Por ejemplo: get_movie_info("http://c3-capstone2.jialonso.com/tt0068646.html") devolvería: 
    {
      'id': 'tt0068646',
      'title': {
          'primary': 'El padrino',
          'original': 'The Godfather'
      },
      'year': 1972,
      'genres': ['Crime', 'Drama'],
      'runtimeMinutes': 175
    }
    
    Podéis utilizar el siguiente código como base:
    
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
            
    _id = url.split('/')[-1].split('.')[0]
    title_wrapper = soup.select_one('div.title_wrapper')  # Toda la información de la película se incluye dentro de este div
    
    title_primary_tag = ??? 
    title_original_tag = ???
    year_tag = ???
    genres_tags = ???
    runtime_tag = ???
            
    return {
        'id': _id,
        'title': {
            'primary': ???
            'original': ???  # Si original no aparece, poned aquí el primario
        },
        'year': ???
        'genres': ???
        'runtimeMinutes': ???  # Usad la función duration_string_to_minutes(...)
    }
    """
    
    ???


<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
data = [get_movie_info(link) for link in links]
print(data)

In [ ]:
assert data[1]["runtimeMinutes"] == 175
assert data[7]["title"]["original"] == "Pulp Fiction"

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

El objeto `data`, en formato lista de diccionarios, lo podemos importar en un dataframe de pandas haciendo uso del método `json_normalize`.

In [ ]:
from pandas import json_normalize

df_webscraping = json_normalize(data)
df_webscraping.set_index('id', inplace=True)
df_webscraping.head()

---

### <font color="#00586D"> 1.2 Listado completo de películas de IMDB. </font>
<br>

Como hemos visto en el apartado anterior, el proceso de web scraping, aunque es posible, es muy lento y además no disponemos de un *dump* completo de IMDB. Por este motivo os proporcionamos un archivo con los datos básicos de más de 400.000 películas. Este archivo contiene películas posteriores al año 1927, es decir, al año en que tuvo lugar la primera proyección de cine con sonido.

El archivo proporcionado es un fichero *zip* que contiene un fichero json por cada año, entre 1928 y 2019. Cada uno de estos ficheros tiene la misma estructura que el objeto `data` obtenido en el apartado anterior.

Podemos descargar y descomprir el fichero *zip* directamente desde python:

In [4]:
import io
import json
import os
import zipfile

if not os.path.exists('datos/imdb/movies/'):
    url = 'http://c3-capstone2.jialonso.com/data/movies-json.zip'
    res = requests.get(url)
    obj = io.BytesIO(res.content)

    with zipfile.ZipFile(obj) as zip_ref:
        zip_ref.extractall('datos/imdb/movies/')

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 3</b></font>

El siguiente paso es leer estos ficheros JSON y crear un único dataframe con la información de todas las películas.

In [ ]:
def get_basics_df():
    """
    Este método devuelve un dataframe con los datos de todas las películas contenidas en el fichero
    movies-json.zip
    """
    ???
    

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
df_basics = get_basics_df()
df_basics.head()

In [ ]:
assert set(df_basics.columns) == {"year", "runtimeMinutes", "genres", "title.original", "title.primary"}
assert len(df_basics) == 415633

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

Las características del dataframe resultante son las siguientes:

In [ ]:
df_basics.info()

---

### <font color="#00586D"> 1.3 Carga del fichero de ratings. </font>
<br>

_IMDB_ pone a disposición pública algunos conjuntos de datos actualizados en formato `csv` [(enlace)](https://datasets.imdbws.com). En este proyecto utilizaremos el fichero `title.ratings.gz` que contiene información sobre las valoraciones hechas a cada película. Nótese también la existencia del fichero `title.basics.tsv.gz` que contiene información muy similar a la proporcionada en el apartado anterior.

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 4</b></font>

Leed el conjuntos de datos de ratings y almacenarlo en un _DataFrame_ llamado `df_ratings`. Para ello, considerar que las columnas están separadas por tabulaciones (caracter `"\t"`).  Utilizar la primera columna como índice.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: _Pandas_ también permite leer directamente archivos comprimidos directamente. Para ello, en este caso, hay que pasarle el parámetro `compression='gzip'`.
</div>


In [ ]:
def get_df_ratings():
    """
    Devuelve el dataframe df_ratings...
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
df_ratings = get_df_ratings()
df_ratings.head()

In [ ]:
assert set(df_ratings.columns) == {"averageRating", "numVotes"}

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

Las características del dataframe resultante son las siguientes:

In [ ]:
df_ratings.info()

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: La diferencia en el número de entradas entre los dataframes de información básica y el de ratings se debe a que el segundo incluye también información de series, cortos, etc.
</div>

---

### <font color="#00586D"> 1.4 Creación del conjunto de datos inicial </font>
<br>

Puesto que el número de títulos sigue siendo elevado, se van a considerar solamente aquellos títulos que han recibido más atención. 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> </font> Seleccionar los títulos que han recibido más (estrictamente) de 50000 votos. 

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Incluid los estrictamente posteriores, es decir, no se incluyen los que han recibido 50000 votos (por operatividad en casos posteriores).
</div>

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 5</b></font>

In [ ]:
def filter_df_ratings(min_votes):
    """
    Filtra el dataframe df_ratings y devuelve un nuevo dataframe en el que todas las filas tienen al menos min_votes
    """
    ???
    

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
df_ratings = filter_df_ratings(50000)
df_ratings.head()

In [ ]:
assert df_ratings["numVotes"].min() > 50000

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

Una vez acotados ambos conjuntos de datos se creará un solo conjunto, al que se denominará `df_movies`, y que contendrá las películas que aparezcan en ambos, es decir:

* películas posteriores a 1927 (datos iniciales proporcionados en el apartado 1.1)
* han rebido más de 50000 votos.

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 6</b></font>

Unir los dos conjuntos de datos anteriores y almacenarlos en el _DataFrame_ `df_movies`. 

In [ ]:
def join_dataframes():
    """
    Une los dataframes df_basics y df_ratings y devuelve el dataframe resultante
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
print("El número final de títulos es {:d}".format(len(df_ratings)))

df_movies = join_dataframes()
df_movies.head()

In [ ]:
assert set(df_movies.columns) == {"year", "runtimeMinutes", "genres", "title.original", "title.primary", "averageRating", "numVotes"}

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

Por último, se conservarán los datos relevantes (algunos datos descartados se actualizarán posteriormente).

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 7</b></font>

Conservar solamente las columnas `'title.primary'`, `'genres'`, `'averageRating'` y `'numVotes'` de `df_movies`. Renombrarlas como `'title'`,`'genres'`,`'avg_rating'`y `'num_votes`. Renombrar el índice como `movie_id `.

In [ ]:
def select_columns():
    """
    Selecciona las columnas del dataframe df_movies y devuelve el nuevo dataframe
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
df_movies = select_columns()
df_movies.head()

In [ ]:
assert set(df_movies.columns) == {"title", "genres", "avg_rating", "num_votes"}
assert df_movies.index.name == "movie_id"

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

Para ilustrar algunos ejemplos en el proyecto, se utilizará la película con identificador`'tt0068646'`: ___The Godfather___.

In [ ]:
tgf_movie_id = 'tt0068646'
df_movies.loc[tgf_movie_id]

<div style="text-align: right">
<a href="#inicio"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---


<a id="section2"></a>
## <font color="#00586D"> 2. The Movie Database (TMDb)  </font>

<br>

_The Movie Database ([TMDB](https://www.themoviedb.org))_ es otro recurso en la web que contiene información sobre películas y series, y se planteó como una alternativa a _IMDb_. A día de hoy, su uso como medio de recogida de críticas y valoraciones _es prácticamente nulo_. Sin embargo,  contiene información completa y actualizada sobre películas, y proporciona una API REST muy completa. En este proyecto utilizaremos la API de _TMDb_ para ampliar el conjunto inicial de datos. 

La API de _TMDb_ requiere auntentificación, por lo que para trabajar con ella es necesario, en primer lugar, disponer de un usuario. Una vez hecho el registro en el sitio, es necesario solicitar una clave para el uso de la API. Las instrucciones detalladas se muestran en esta página ([enlace](https://developers.themoviedb.org/3/getting-started/introduction)). Este proceso es sencillo, y básicamente consiste en 3 pasos:

1. Entrar en la configuración de la cuenta personal.
2. Entrar en el menú de la API.
3. Crear la API. 

Posteriormente es posible acceder, dentro de la configuración de la cuenta, a la sección correspondiente a la API. En ella se encuentran varios tokens de acceso. En este proyecto necesitamos el primero de ellos, etiquetado como _Clave de la API (v3 auth)_. Copiar este token y asignarlo a la variable `token`.

In [ ]:
token = ???

La API de _TMDb_ es accesible a través de `https://api.themoviedb.org/3/`. En la documentación ([enlace](https://developers.themoviedb.org/3/getting-started/introduction)) pueden consultarse todos los _endpoints_ que proporciona. Además, desde este sitio es posible probar los métodos, y generar las llamadas en diversos lenguajes. En el caso de _Python_ se generan dos alternativas. La primera utiliza la librería `http.client`, mientras que la segunda utiliza `requests`.

---

### <font color="#00586D"> 2.1. Obtención de información sobre cada película.</font>

<br>

La llamada `GET /movie/{movie_id}` permite obtener información sobre una película. El campo `movie_id` corresponde al identificador de la película, que puede ser el propio de _TMDb_, o el identificador en _IMDb_ (que es del que disponemos). Uno de los parámetros que toma  es `language`, que en este proyecto se fijará a `en_US`. 

In [ ]:
import requests
import json

language = 'en_US'  # 'es_ES' para español.
params = {'api_key':token, 'languaje':language}

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 8</b></font>

Obtener la información relativa la película _The Godfather_  (cuyo código es `'tt0068646'` y está almacenado en `tgf_movie_id`).

In [ ]:
def get_one_movie(movie_id):
    """
    Devuelve un diccionario con los datos de la película recibida como parámetro del API de TMDB
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
tgf_data = get_one_movie(tgf_movie_id)

# Esta es una de las formas de mostrar un json en formato "amigable"
print(json.dumps(tgf_data, indent=3))

In [ ]:
assert tgf_data["release_date"] == "1972-03-14"

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: El campo `poster_path` del objeto JSON contiene el identificador del poster de la película, que puede ser accedido como `http://image.tmdb.org/t/p/w185/poster_path`.  
</div>


In [ ]:
from IPython.display import Image
Image('http://image.tmdb.org/t/p/w185'+tgf_data['poster_path'])

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 9</b></font>

Una vez implementado el acceso a los datos de una película mediante la API de _TMDb_, se obtendrán los datos relativos a todas las películas referenciadas en `df_movies`, y se almacenarán en una estructura JSON que se denominará `movie_data`.

<div class="alert alert-block alert-danger">
    
<i class="fa fa-exclamation-triangle" aria-hidden="true"></i> __Muy importante:__ Cada llamada a la API requiere aproximadamente 0.5 segundos. Por tanto, la obtención de los datos para todas las películas puede requerir, si se hace secuencialmente, entorno a 1500 segundos. Debido a esto, y de cara a facilitar el desarrollo del trabajo en distintas sesiones, es conveniente almacenar los resultados en un archivo, al que se denominará `datos/backup/movie_data.json`. Una vez disponible este archivo, en las sesiones posteriores de trabajo, los datos se leerán de aquí.
</div>

In [ ]:
def get_movie_data():
    """
    Obtiene y devuelve la estructura movie_data con el formato:
    {
      "imdb_id1": ...,
      "imdb_id2": ...,
    }
    
    Por ejemplo:
    {
      "tt0068646": {
         "adult": False,
         "backdrop_path": "/rSPw7tgCH9c6NqICZef4kZjFOQ5.jpg",
         ...
      },
      ...
    }
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
import os

# Si no se han descargado todavía los datos, los descarga y almacena en el archivo.
if not os.path.isfile('datos/backup/movie_data.json'):
    # Descarga los datos
    movie_data = get_movie_data()
    
    with open('datos/backup/movie_data.json',"w") as movie_data_file:
        json.dump(movie_data, movie_data_file)    
        
# Si se habían descargado anteriormente, y el archivo está disponible, los lee. 
else:
    with open('datos/backup/movie_data.json','r') as movie_data_file:
        movie_data = json.load(movie_data_file)
    
    
print("El de títulos obtenidos es {:d}".format(len(movie_data)))

In [ ]:
assert movie_data[tgf_movie_id]["release_date"] == "1972-03-14"

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

---

### <font color="#00586D"> 2.2. Incorporación de información de interés al conjunto de datos.</font>

<br>

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 10</b></font>

Una vez obtenidos los datos relativos a cada película, y almacenados en la estructura `movie_data`, se ha de incorporar la información relevante  al _DataFrame_ `df_movies`. En concreto, se crearán columnas que contengan, para cada película, los siguientes campos (entre paréntesis se dan los tipos que han de utilizarse): 

* `budget (integer)`, que corresponde al presupuesto,
* `revenue (integer)`, que corresponde a las ganancias,
* `release_date (TimeStamp)`, que corresponde a la fecha de estreno,
* `original_language (String)`, que corresponde al idioma original,
* `popularity (float)`, que corresponde a la popularidad.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> Existen varias formas de llevar a cabo esta incorporación de datos. Una de ellas es crear una serie para cada una de las columnas mediante datos obtenidos con comprensión de listas, e incorporarla al _DataFrame_. Otra alternativa consiste en crear un _DataFrame_ con `pd.DataFrame.from_dict` y combinarlo con `df_movies`. 
</div>

In [ ]:
def extend_df_movies():
    """
    Añade las columnas budget, revenue, release_date, original_language y popularity al dataframe df_movies y devuelve el dataframe resultante
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
df_movies = extend_df_movies()
df_movies.head()

In [ ]:
assert set(df_movies.columns) == {"title", "genres", "avg_rating", "num_votes", "budget", "revenue", "release_date", "original_language", "popularity", "poster_path"}
df_movies.info()

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

---

### <font color="#00586D">2.3. Palabras clave</font>

<br>

La llamada `https://api.themoviedb.org/3/movie/'+movie_id+'/keywords` devuelve las palabras clave correspondientes a la película con identificador `movie_id`. 

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 11</b></font>

Obtener y mostrar las palabras claves para la película _The Godfather_ (el código está almacenado en la variable `tgf_movie_id`).

In [ ]:
def get_one_movie_keywords(movie_id):
    """
    Devuelve los keywords con los datos de la película recibida como parámetro del API de TMDB
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
tgf_data = get_one_movie_keywords(tgf_movie_id)

print(json.dumps(tgf_data,indent=3))

In [ ]:
assert any([keyword["name"] == "mafia" for keyword in tgf_data["keywords"]])

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 12</b></font>

Obtener las palabras clave de cada una de las películas y almacenarlas en un objeto JSON denominado `keywords_data`. 

<div class="alert alert-block alert-danger">
    
<i class="fa fa-exclamation-triangle" aria-hidden="true"></i> Al igual que en el caso anterior, se guardarán los datos en un archivo, denominado ahora `datos/backup/keywords_data.json` para evitar tener que llamar a la API en sesiones de trabajo posteriores. 
</div>

In [ ]:
def get_keywords_data():
    """
    Obtiene y devuelve la estructura keywords_data
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
if not os.path.isfile('datos/backup/keywords_data.json'):
    keywords_data = get_keywords_data()
    
    with open('datos/backup/keywords_data.json',"w") as keywords_data_file:
        json.dump(keywords_data, keywords_data_file)    
        
# Si se hanían descargado anteriormente, los lee. 
else:
    with open('datos/backup/keywords_data.json','r') as keywords_data_file:
        keywords_data = json.load(keywords_data_file)   
        
print("El de títulos obtenidos es {:d}".format(len(keywords_data)))        

In [ ]:
assert any([keyword["name"] == "mafia" for keyword in keywords_data[tgf_movie_id]["keywords"]])

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

In [ ]:
len(df_movies)

El formato con que se devuelven las palabras clave para cada película tiene varios niveles. Se trata de un diccionario con un campo denominado `keywords` que contiene la lista con las palabras clave. Cada una representada, su vez por un diccionario del que interesa el campo `name`. 

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 13</b></font>

Añadir a `df_movies` una columna, denominada `keywords` que contenga la lista de palabras clave para cada película. Este dato se obtendrá de `keywords_data`.

<div class="alert alert-block alert-danger">
    
<i class="fa fa-exclamation-triangle" aria-hidden="true"></i> Este ejercicio es difícil y requiere, sobre todo, tener clara la estructura. Una opción consiste en utilizar `df.apply(,axis=1)`.
</div>

In [ ]:
def get_keywords_serie():
    """
    Obtiene y devuelve la serie de datos keywords
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
df_movies['keywords'] = get_keywords_serie()
df_movies.head()

In [ ]:
assert 'mafia' in df_movies.loc[tgf_movie_id].keywords

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

<div style="text-align: right">
<a href="#inicio"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<a id="section4"></a>
## <font color="#00586D"> 3. MongoDB </font>

<br>

A continuación vamos a almacenar el resultado de todo el proceso de extracción en un formato en el que podamos persistir y consultar los datos de manera cómoda como es mongoDB

In [ ]:
import pymongo

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 14</b></font>

Crea un cluster de mongodb en Mongodb Atlas y abre una conexión utilizando pymongo. A continuación crea una base de datos llamada `capstone` y dentro una colección llamada `movies`.

In [ ]:
def create_mongodb_connection():
    """
    Establece la conexión con el cluster de Mongodb Atlas
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
mongoClient = create_mongodb_connection()

db = mongoClient.capstone
movies = db.movies

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 15</b></font>

Inserta el contenido de la base de datos `df_movies` en la colección de mongo. Para ello deberás exportar los datos a un diccionario utilizando la función `to_dict`. Recuerda que el formato viene determinado por el parámetro `orient`, si tienes dudas puedes consultar la documentación utilizando el comando `??df_movies.to_dict`

In [ ]:
def insert_data():
    """
    Inserta los datos en Mongodb
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
insert_data()
assert movies.estimated_document_count() > 0

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 16</b></font>

Vamos a probar a hacer una búsqueda muy sencilla. Intenta encontrar las películas con un rating superior a 9:

In [ ]:
def get_movies_by_rating(rating):
    """
    Obtiene las películas con un rating superior al recibido como parámetro
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
cursor = get_movies_by_rating(9)

for m in cursor[0:10]:
    print(m["title"])

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 17</b></font>

Ahora encuentra las películas cuyo género es `Action`

In [ ]:
def get_movies_by_genre(genre):
    """
    Obtiene las películas del género recibido como parámetro
    """
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
cursor = get_movies_by_genre("Action")

for m in cursor[0:10]:
    print(m["title"])

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> EJERCICIO 18</b></font>

Implementa una función que:

- Reciba un input de usuario con una lista de keywords separadas por comas
- Convierta el contenido a una lista de keywords (usa split)
- Busca en mongoDB las películas que contengan esas keywords
- Extrae las primeras 20 portadas
- Imprime las portadas utilizando la utilidad Image

In [2]:
from IPython.display import Image

In [ ]:
def movies_by_keywords(keywords):
    ???

<font color="#00586D" size=3><b><i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> VALIDACIÓN</b></font>

In [ ]:
movies_by_keywords("war,love")

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

<div style="text-align: right">
<a href="#inicio"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#00586D"></i> </font></div>